# Introduction To Monte Carlo Reinforcement Learning

## CSCI E-82A

## Stephen Elston

Starting with this lesson, we will turn our attention to a **reinforcement learning**. Reinforcement learning is a distinctive type of machine learning, differing from supervised learning and unsupervised learning. 

Reinforcement learning has several characteristics, which differentiate this method from other machine learning and from dynamic programming. The table below outlines key differences between the model types:


| Model Type | Environment Model | State | Labeled Data | Loss Function |
| :--- | :---: | :---: | :---: | :----- |
|Supervised Learning | Yes | No | Yes| Error metric |
|Unsupervised Learning | Yes | No | No | Error metric |
| Bandit Agent | No | No | No | Reward |
| Dynamic Programming | Yes | Yes | No | Reward |
| Reinforcement Learning | No | Yes | No | Reward | 

Some highlights of these differences include:  


- Like dynamic programming, reinforcement learning **optimizes a reward function**. This is in contrast to supervised and unsupervised learning which attempt to minimize error or loss function.  
- **No Markov model** needs to be specified for reinforcement learning, in contrast to dynamic programming.
- Reinforcement learning algorithms learn by **experience**. Over time, the algorithm learns a model of the environment and these results are used to optimize the expected reward. Learning from experience is in contrast to supervised learning which uses marked cases (labels).  
- Reinforcement learning agents take **actions** and only receive **state** and **rewards** from the environment. These are the only interaction between the RL agent and the environment.   

The interaction between a reinforcement learning agent and the environment are illustrated in the figure below. Notice that the only feedback the agent receives from the environment is the reward and and state information. The agent receives no other evidence.   

<img src="img/RL_AgentModel.JPG" alt="Drawing" style="width:500px; height:300px"/>
<center> **Reinforcement Learning Agent and Environment** </center>  


**Suggested readings** for Monte Carlo reinforcement learning Chapter 5 of Sutton and Barto, second edition, provides a good introduction, including many alternative algorithms not discussed here.   

## Overview of Monte Carlo Reinforcement Learning

A wide variety of reinforcement learning algorithms have been developed over the past few decades. In this lesson we will explore the basics of the Monte Carlo method. Monte Carlo algorithms have been known for most of the history of reinforcement learning. However, they are generally considered inefficient for several reasons that will become apparent as we proceed:
1. Monte Carlo methods rely large numbers of **random samples** to produce estimates. Thus, Monte Carlo algorithms are inherently computationally intensive. 
2. Monte Carlo reinforcement learning algorithms must **complete an entire episode** before any reward estimate can be produce and the policy improved. 

### Basics of Monte Carlo Simulation

Monte Carlo sampling was developed in the 1940s. Originally, Monte Carlo methods were used to compute estimates of complex functions which where analytically intractable. The basic idea is to **compute an estimate** of a complex function by **averaging a large number of samples**. 

Monte Carlo methods rely on the [**weak law of large numbers**](https://en.wikipedia.org/wiki/Law_of_large_numbers). The law of large numbers is a theorem that states that statistics of independent samples converge to the population values as more unbiased experiments are performed. We can write this mathematically for the **expected value** pr mean as:

$$Let\ \bar{X} = \frac{1}{n}\sum_{i=1}^{n} X_i\\
then\ by\ the\ law\ of\ Large\ Numbers\\
\bar{X} \rightarrow E(X) = \mu\\
as\\
n \rightarrow \infty$$

Thus, if we sample some process $X$ enough times (possibly infinite), we can compute the expected value from these samples. 

### Monte Carlo Reinforcement Learning

But, how do we apply Monte Carlo sampling to reinforcement learning? More specifically, how do we apply Monte Carlo sampling to **episodic** reinforcement learning tasks. 

To understand this algorithm, it helps to examine the backup diagram shown below. This diagram shows Monte Carlo sampling of a single episode.    

<img src="img/MC_Backup.JPG" alt="Drawing" style="width:75px; height:400px"/>
<center> **Backup Diagram for Monte Carlo Reinforcement Learning** </center>  

Starting at the top of the diagram the system is in a state, s. An action, a, causes a transition to a new state. The sampling of the episode proceeds until the terminal state, t, is reached. The return for the initial state can only be computed once the Monte Carlo backup **ends at the terminal state**. In other words, **Monte Carlo algorithms do not bootstrap**.  

In reinforcement learning we do not know the model. But, the agent can take a series of actions and find the rewards for these actions. For each episode the agent will accumulate the history of rewards for each action, given the state. 

Recall that for a finite or episodic Markov reward processes we define the **return** for state transitions starting with the current state. The return is the sum of the rewards for the $T$ future states transitions of the episodic process, and can be expressed as:

$$G_t = R_{t+1} + R_{t+2} + \ldots = R_{T}= \sum_{k = 0}^{T} R_{t+k+1}$$ 

Thus, for any episode the Monte Carlo algorithm will sample the return for the states visited. Over a large (actually infinite) number of episodes the Monte Carlo algorithm will sample each action value several times. The sampled return values are then averaged for each state action. This process will converge to the actual action values, which are **unobservable** directly. 

For sampling a single episode of a Markov process, the Monte Carlo algorithm may or may not visit a state one or more times. The question then becomes, How should returns be computed if a state is visited more than once in an episode? There are two options each of which has different statical convergence properties:
1. **First visit** Monte Carlo estimates returns from rewards from the first visit to a state in an episode. We will use first visit Monte Carlo in this lesson.
2. **Every visit** Monte Carlo accumulates the rewards for any visit to a state in an episode.


## Example of First Visit Monte Carlo RL

With this short introduction MC RL learning in mind, tet's try an example. We will sample the action value function using a simple MC algorithm here. 

**Navigation** to a goal is a significant problem in robotics. Real-world navigation is rather complex. Therefore, in this example we will use a simple analog called a **grid world**. The grid world for this problem is shown below. 

<img src="img/GridWorld.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **A 4x4 Grid World with Terminal State** </center>

The grid world consists of a 4x4 set of positions the robot can occupy. Each position is considered a state. The goal is to navigate to state 0, the goal, in the minimum steps. We will explore methods to find policies which reach this goal and achieve maximum reward. 

Grid position 0 is the goal and a **terminal state**. There are no possible state transitions out of this position. The presence of a terminal state makes this an **episodic Markov random process**. For each episode sampled the robot can start in any other random position, $\{ 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15 \}$. This random selection process makes this a **random start** Monte Carlo algorithm. The episode terminates when the robot enters the terminal position (state 0).  



In reality, an RL agent may need to explore to find the possible actions when it is in some particular state. To simplify our example, we encode, or represent, these possibilities in a dictionary as shown in the code block below. We use a dictionary of dictionaries to perform the lookup. The keys of the outer dictionary are the identifiers (numbers) of the states. The keys of the inner dictionary are the possible actions and the values are the **successor state**, $s'$, for that transition.  

In each state, there are four possible actions the robot can take:
- up, u
- down, d,
- left, l
- right, r

The MC RL agent has no model. Therefore, beyond these allowed actions, all other information is encapsulated in the environment and is unobservable by the agent. This is the key difference between reinforcement learning and dynamic programming. 

In [75]:
## import numpy for latter
import numpy as np
import numpy.random as nr

## Define the transition dictonary of dictionaries:
neighbors = {0:{'u':0, 'd':0, 'l':0, 'r':0},
          1:{'u':1, 'd':5, 'l':0, 'r':2},
          2:{'u':2, 'd':6, 'l':1, 'r':3},
          3:{'u':3, 'd':7, 'l':2, 'r':3},
          4:{'u':0, 'd':8, 'l':4, 'r':5},
          5:{'u':1, 'd':9, 'l':4, 'r':6},
          6:{'u':2, 'd':10, 'l':5, 'r':7},
          7:{'u':3, 'd':11, 'l':6, 'r':7},
          8:{'u':4, 'd':12, 'l':8, 'r':9},
          9:{'u':5, 'd':13, 'l':8, 'r':10},
          10:{'u':6, 'd':14, 'l':9, 'r':11},
          11:{'u':7, 'd':15, 'l':10, 'r':11},
          12:{'u':8, 'd':12, 'l':12, 'r':13},
          13:{'u':9, 'd':13, 'l':12, 'r':14},
          14:{'u':10, 'd':14, 'l':13, 'r':15},
          15:{'u':11, 'd':15, 'l':14, 'r':15}}

To simulate the environment, we need a reward structure. In this case, the robot receives the following rewards:   

- 10 for entering position 0. 
- -1 for attempting to leave the grid. In other words, we penalize the robot for hitting the edges of the grid.  
- -0.1 for all other state transitions, which is the cost for the robot to move from one state to another. If we did not have this penalty, the robot could follow any random plan to the goal which did not hit the edges. 

This **reward structure is unknown to the MC RL agent**. The agent must **learn** the rewards by sampling the environment. Here the rewards are in the form of action values.    

We encode these rewards in the same type of dictionary structure used for the foregoing structures. 

In [76]:
rewards = {0:{'u':10.0, 'd':10.0, 'l':10.0, 'r':10.0},
          1:{'u':-1, 'd':-0.1, 'l':10.0, 'r':-0.1},
          2:{'u':-1.0, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          3:{'u':-1.0, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          4:{'u':10.0, 'd':-0.1, 'l':-1.0, 'r':-0.1},
          5:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          6:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          7:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          8:{'u':-0.1, 'd':-0.1, 'l':-1.0, 'r':-0.1},
          9:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          10:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          11:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          12:{'u':-0.1, 'd':-1.0, 'l':-1.0, 'r':-0.1},
          13:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-0.1},
          14:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-0.1},
          15:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-1.0}}

With the properties of the environment defined, it is time to create an environment simulator. The code in the cell below simulates the environment. The function is called with a state and action. It returns the next state, 's_prime' and 'reward'. 

To simplify the rest of the code in this notebook we are treating the dictionaries as global. In general, this would be considered poor programming practice. 

Execute the code in the cell below and observe the results from the test cases. 

In [77]:
def simulate_environment(s, action, neighbors = neighbors, rewards = rewards, terminal = 0):
    """
    Function simulates the environment
    returns s_prime and reward given s and action
    """
    s_prime = neighbors[s][action]
    reward = rewards[s][action]
    return (s_prime, reward, is_terminal(s_prime, terminal))

def is_terminal(state, terminal = 0):
    return state == terminal

## Test the function
for a in ['u', 'd', 'r', 'l']:
    print(simulate_environment(1, a))

(1, -1, False)
(5, -0.1, False)
(2, -0.1, False)
(0, 10.0, True)


These results look correct. It appears the simulator works correctly. 

## MC Policy Evaluation

**Policy evaluation** is an essential part of reinforcement learning. Policy evaluation is required to compare the performance of different reinforcement learning methods. Policy evaluation is performed using state-value estimation methods. In this case, we will use a Monte Carlo state-value estimation algorithm.   

To develop and test a policy evaluation algorithm, we need to define the transition probabilities for an initial policy. We set the probabilities for each transition as a **uniform distribution** leading to random action by the robot. As there are 4 possible transitions from each state, this means all transition probabilities are 0.25. In other words, this is a random policy which does not favor any particular plan. 

The initial uniform transition probabilities are encoded using a dictionary of dictionaries. The organization of this data structure is identical to the foregoing data structure. 

In [78]:
initial_policy = {0:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        1:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25}, 
                        2:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        3:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        4:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        5:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        6:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        7:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        8:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        9:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        10:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        11:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        12:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        13:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        14:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        15:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25}}

For policy evaluation We are using random start Monte Carlo starts. The code in the cell below generates a random state to start the Monte Carlo episode, which is not the terminal state. Execute this code and examine the results pf the test cases. 

In [79]:
def start_episode(n_states):
    '''Function to find a random starting value for the episode
    that is not the terminal state'''
    state = nr.choice(range(n_states))
    while(is_terminal(state)):
         state = nr.choice(range(n_states))
    return state

## test the function to make sure never starting in terminal state
[start_episode(15) for _ in range(10)]

[3, 6, 10, 12, 2, 3, 11, 7, 1, 8]

The function in the cell below finds an action given the state and the policy by the following steps:  
1. The probability of taking an action and is determined by the transition probabilities specified by the policy. 
2. The next state and the reward are received by making queries to the environment using the `simulate_environment` function. 

Notice that the agent receives no specific information about the environment.   

Execute this code and examine the results of the test cases.

In [80]:
def take_action(state, policy, actions = {1:'u', 2:'d', 3:'l', 4:'r'}):
    '''Function takes action given state using the transition probabilities 
    of the policy'''
    ## Find the action given the transistion probabilities defined by the policy.
    action = actions[nr.choice(range(len(actions)), p = list(policy[state].values())) + 1]
    s_prime, reward, is_terminal = simulate_environment(state, action)
    return (action, s_prime, reward, is_terminal)

## Test function for several states
for s in range(16):
    print(take_action(s, initial_policy))

('u', 0, 10.0, True)
('r', 2, -0.1, False)
('d', 6, -0.1, False)
('r', 3, -1.0, False)
('u', 0, 10.0, True)
('d', 9, -0.1, False)
('d', 10, -0.1, False)
('d', 11, -0.1, False)
('u', 4, -0.1, False)
('l', 8, -0.1, False)
('l', 9, -0.1, False)
('l', 10, -0.1, False)
('d', 12, -1.0, False)
('r', 14, -0.1, False)
('l', 13, -0.1, False)
('r', 15, -1.0, False)


We are now have all the prerequisites for the functions which do most of the work for policy evaluation. 

The first function is called `MC_episode`, which performs first visit Monte Carlo for one episode by the following steps:
1. A loop is executed until episode ends when a terminal state is reached. 
2. The state transitions are determined using the policy with the `take_action` function.  
3. The function accumulates the reward and and number of times a state has been visited.  

The `MC_state_values` function computes the state values for a number of episodes by the following steps:

1.  Iterates over the specified number of episodes. For each episode the `MC_episode` function is called and the reward accumulated. 
2. Once the episodes have concluded, the state values are normalized by dividing the accumulated rewards by the number of visits.

Execute the code and examine the resulting state-values for the random walk policy. 

In [81]:
def MC_episode(policy, G, n_visits, episode, n_states): 
    '''Function creates the Monte Carlo samples of one episode.
    This function does most of the real work'''
    ## For each episode we use a list to keep track of states we have visited.
    ## Once we visit a state we need to accumulate values to get the returns
    states_visited = []
    states = list(policy.keys())
        
    ## Find the starting state
    current_state = start_episode(n_states)
    terminal = False
    g = 0.0
        
    while(not terminal):
        ## Find the next action and reward
        action, s_prime, reward, terminal = take_action(current_state, policy)
            
        ## Add the reward to the states visited if this is a first visit  
        if(current_state not in states_visited):
            ## Mark that the current state has been visited 
            states_visited.append(current_state) 
            ## Add the reward to states visited 
            for state in states_visited:
                n_visits[state] = n_visits[state] + 1.0
                G[state] = G[state] + (reward - G[state])/n_visits[state]
        
        ## Update the current state for next transition
        current_state = s_prime 
    return (G, n_visits) 
    
    


def MC_state_values(policy, n_episodes):
    '''Function that evaluates the state value of 
    a policy using the Monte Carlo method.'''
    ## Create list of states 
    states = list(initial_policy.keys())
    n_states = len(states)
    
    ## An array to hold the accumulated returns as we visit states
    G = np.zeros((n_states))
    
    ## An array to keep track of how many times we visit each state so we can 
    ## compute the mean
    n_visits = np.zeros((n_states))
    
    ## Iterate over the episodes
    for i in range(n_episodes):
        G, n_visits = MC_episode(policy, G, n_visits, i, n_states) # neighbors, i, n_states)
    return(state_values) 
        

## Test the functions
nr.seed(335)
state_values = MC_state_values(initial_policy, n_episodes = 2000)
print(state_values.reshape((4,4)))

[[0.         5.57434357 2.76479076 2.04389007]
 [5.65890318 2.78748107 2.01472932 1.59934829]
 [2.22452534 1.88735317 1.52328405 1.24284272]
 [1.81315597 1.47838912 1.2324538  1.06050072]]


These results look promising. The returns become smaller the further the state is from the goal. 

## Policy Improvement

Now that we have a way to evaluate a policy using a first visit Monte Carlo algorithm we need a method to improve policy. Monte Carlo policy improvement employs the following steps:
1. Sample rewards given state, s, and action a, following policy $\pi$. 
2. The action values are computed from returns. 
2. The policy is updated using an $\epsilon$-greedy algorithm. 

The first function, `MC_action_value_episode`, required computes the action-values given the current policy for a single episode. This function uses a first visit Monte Carlo algorithm to find the action-values for a single episode with the following steps:

1. The `take_action` function is used to determine the next action given the state following the policy. 
2. The visit is recoded in the `state_actions_visited` array. 
3. The core of the function are a pair of nested loops over states and actions. For states that have been visited, the reward is summed and the number of visits is incremented.   

The numpy array, `Q`, holds the accumulated reward. The row index is for state and the column index is for action, $\{ up,\ down,\ left,\ right \}$. Th|e numpy array `state_actions_visited` is indexed in the same manner.

Execute this code and examine the results of the basic test cases. 


In [93]:
import itertools
def MC_action_value_episode(policy, Q, n_visits, inital_state, n_states, n_actions, action_index = {'u':0, 'd':1, 'l':2, 'r':3}):
    '''Function creates the Monte Carlo samples of action values for one episode.
    This function does most of the real work'''
    ## For each episode we use a list to keep track of states we have visited.
    ## Once we visit a state we need to accumulate values to get the returns
    state_actions_visited = np.zeros((n_states, n_actions))
    
    current_state = initial_state
    terminal = False  
    while(not terminal):
        ## Find the next action and reward
        action, s_prime, reward, terminal = take_action(current_state, policy)

        action_idx = action_index[action]         
        
        ## Check if this state-action has been visited.
        if(state_actions_visited[current_state, action_idx] != 1.0):
            ## Mark that the current state-action has been visited 
            state_actions_visited[current_state, action_idx] = 1.0  
            ## This is first vist MS, so must loop over all state-action pairs and 
            ## add the reward and increment the count for the ones visited.
            for s,a in list(itertools.product(range(n_states), range(n_actions))):
                ## Add reward to if these has been a visit to the state
                if(state_actions_visited[s,a] == 1.0):
                    n_visits[s,a] = n_visits[s,a] + 1.0
                    Q[s,a] = Q[s,a] + (reward - Q[s,a])/n_visits[s,a]    
        ## Update the current state for next transition
        current_state = s_prime
    return (Q, n_visits) 

## Basic test of the function
n_actions = 4
n_states = 16
Q = np.zeros((n_states, n_actions))
n_visits = np.zeros((n_states, n_actions))
initial_state = 15
Q, n_visits = MC_action_value_episode(initial_policy, Q, n_visits, initial_state, n_states, n_actions)
print('Q')
print(Q)
print('\nn_visits')
print(n_visits)

Q
[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [10.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 4.95        0.          0.          0.        ]
 [ 0.          0.          3.26666667  0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          1.92      ]
 [ 2.425       0.          1.58333333  0.        ]
 [ 0.          1.05        1.34285714  0.        ]
 [ 0.          0.          0.92222222  0.        ]]

n_visits
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 3. 0.]
 [0. 0. 0. 0.]
 [0. 0

The function in the cell below computes the action values using the specified number of episodes, using first visit Monte Carlo. The steps are:

1. For each episode, the `MC_action_value_episode` function is called. 
2. The accumulated rewards are normalized by the number of visits to compute the action values. 

Execute this code and examine the results. 

In [94]:
def print_Q(Q):
    Q = pd.DataFrame(Q, columns = ['up', 'down', 'left', 'right'])
    print(Q)

def MC_action_values(policy, Q, n_episodes, inital_state):
    '''Function evaluates the action-values given a policy for the specified number of episodes and 
    initial state'''
    n_states = len(policy)
    n_actions = len(policy[0])
    ## Array to count visits to action-value pairs
    n_visits = np.zeros((n_states, n_actions))
    ## Dictionary to hold neighbor states
    neighbors = {}
    
    ## Loop over number of episodes
    for _ in range(n_episodes):
        ## One episode of MC
        Q, n_visits = MC_action_value_episode(policy, Q, n_visits, initial_state, n_states, n_actions)
    return(Q)
    
## Basic test of the function
n_episodes = 1000
initial_state = 15
Q = np.zeros((n_states, n_actions))
initial_state = 15
Q = MC_action_values(initial_policy, Q, n_episodes, initial_state)
print_Q(Q)  

[[ 0.          0.          0.          0.        ]
 [ 3.57052632  2.43333333 10.          3.13918919]
 [ 2.10337079  1.64219653  3.48073555  1.14419643]
 [ 0.91688312  1.38        1.55333333  1.39047619]
 [10.          2.65591398  3.27257525  2.41083521]
 [ 3.70247934  1.84874372  3.60513196  1.58318584]
 [ 1.9759799   1.4252      2.02717391  1.11276596]
 [ 1.14324324  1.125       1.58410959  1.34324324]
 [ 3.45892857  1.77863071  2.1031401   1.52708333]
 [ 2.22318841  1.35928144  2.08611111  1.16832579]
 [ 1.47220903  1.10419847  1.64965986  1.17649573]
 [ 1.17349398  0.92406015  1.22119725  0.89594595]
 [ 2.06716229  1.48298279  1.47825465  1.48818898]
 [ 1.62674591  1.15032258  1.50510708  1.22557143]
 [ 1.15139073  1.00937099  1.21923211  0.98679962]
 [ 1.00664093  0.80046512  0.99974737  0.82717557]]


Now that we have the code required to perform first visit Monte Carlo action value estimation, its time to perform policy improvement. 

The `update_policy` function in the cell below performs $\epsilon$-greedy policy improvement given the action values. The steps are:

1. Loop over all states to update the policy for each state. 
2. For each state find the actions with the maximum action value. The best action may not be unique. 
3. The transition probability is computed for the actions with the largest action value. All other actions will be assigned a transition probability of $\epsilon$.
4. The policy is updated with the new action values. 

Execute this code and examine the results. 

In [95]:
def update_policy(policy, Q, epsilon, action_index = {'u':0, 'd':1, 'l':2, 'r':3}):
    '''Updates the policy based on estiamtes of Q using 
    an epslion greedy algorithm. The action with the highest
    action value is used.'''
    
    ## Find the keys for the actions in the policy
    keys = list(policy[0].keys())
    
    ## Iterate over the states and find the maximm action value.
    for state in range(len(policy)):
        ## First find the index of the max Q values  
        q = Q[state,:]
        max_action_index = np.where(q == max(q))[0]

        ## Find the probabilities for the transitions
        n_transitions = float(len(q))
        n_max_transitions = float(len(max_action_index))
        p_max_transitions = (1.0 - epsilon *(n_transitions - n_max_transitions))/(n_max_transitions)
  
        ## Now assign the probabilities to the policy as epsilon greedy.
        for key in keys:
            if(action_index[key] in max_action_index): policy[state][key] = p_max_transitions
            else: policy[state][key] = epsilon
    return(policy)                

update_policy(initial_policy, Q, 0.1)    

{0: {'d': 0.25, 'l': 0.25, 'r': 0.25, 'u': 0.25},
 1: {'d': 0.1, 'l': 0.7, 'r': 0.1, 'u': 0.1},
 2: {'d': 0.1, 'l': 0.7, 'r': 0.1, 'u': 0.1},
 3: {'d': 0.1, 'l': 0.7, 'r': 0.1, 'u': 0.1},
 4: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 5: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 6: {'d': 0.1, 'l': 0.7, 'r': 0.1, 'u': 0.1},
 7: {'d': 0.1, 'l': 0.7, 'r': 0.1, 'u': 0.1},
 8: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 9: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 10: {'d': 0.1, 'l': 0.7, 'r': 0.1, 'u': 0.1},
 11: {'d': 0.1, 'l': 0.7, 'r': 0.1, 'u': 0.1},
 12: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 13: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 14: {'d': 0.1, 'l': 0.7, 'r': 0.1, 'u': 0.1},
 15: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7}}

## Generalized Policy Improvement

When performing MC RL, we can start with an arbitrary initial policy. The MC RL agent will then improve this policy and in the process will **learn the Markov process model**. Again, this is a key difference with dynamic programming where this model is specified. 

RL MC uses the idea of **generalized policy improvement**. Recall that GPI divides the policy improvement and evaluation steps into opposing processes and iterates between them. This process can be done in a quite granular way, even on a single episode or even state at a time. The figure below illustrates the concept of GPI.  

<img src="img/GPI.JPG" alt="Drawing" style="width:250px; height:250px"/>
<center> **Concept of Generalized Policy Improvement** </center>

The code in the cell below uses the GPI method to improve the policy for the grid world using GPI. The outer loop or cycle performs one iteration of GPI which has two steps: 
1. At the start of the loop the returns for the current policy are evaluated. In this case we use the average of several MC episodes. Keep in mind that the agent's walk is determined by the policy, but the returns accumulated are from the environment. 
2. Next, the policy is updated using the new return values. The policy is improved by increasing the transition probabilities for actions (transitions) with higher reward. To ensure that the algorithm **continues to explore**, transition probabilities are never set to 0 but rather to a small minimum value $\epsilon$. 
 
Addition details on the operation of this algorithm can be obtained by reading the comments. Execute this code and examine the results.  

In [85]:
def MC_policy_improvement(policy, n_episodes, n_cycles, inital_state = 15, epsilon = 0.1, n_actions = 4):
    '''Function perfoms GPI using Monte Carlo value estimation.
    Updates to policy are epsilon greedy to prevent the algorithm
    from being trapped at some point.'''
    Q = np.zeros((len(policy), n_actions))
    ## Iterate over the required number of cycles
    for _ in range(n_cycles):
        Q = MC_action_values(policy, Q, n_episodes, inital_state)
        policy = update_policy(policy, Q, epsilon = epsilon)
    return(policy)

improved_policy = MC_policy_improvement(initial_policy, 5000, 5, epsilon = 0.1)  
for state in range(16):
    print(improved_policy[state])

{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}


The improved policy makes sense. Transitions that move the robot closer to the goal are favored. 

Finally, execute the code in the cell below to compute the returns for the improved policy. 

In [86]:
nr.seed(369)
state_values = MC_state_values(improved_policy, n_episodes = 10000)
print(state_values.reshape((4,4)))

[[0.         5.57434357 2.76479076 2.04389007]
 [5.65890318 2.78748107 2.01472932 1.59934829]
 [2.22452534 1.88735317 1.52328405 1.24284272]
 [1.81315597 1.47838912 1.2324538  1.06050072]]


These returns are significantly higher than for the random policy, indicating the policy is indeed an improvement. 

In [87]:
improved_policy_2 = MC_policy_improvement(initial_policy, 100, 5000, epsilon = 0.1)  
for state in range(16):
    print(improved_policy_2[state])

{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.1, 'd': 0.7, 'l': 0.1, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.7, 'd': 0.1, 'l': 0.1, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}
{'u': 0.1, 'd': 0.1, 'l': 0.7, 'r': 0.1}


In [88]:
nr.seed(369)
state_values = MC_state_values(improved_policy_2, n_episodes = 10000)
print(state_values.reshape((4,4)))

[[0.         5.57434357 2.76479076 2.04389007]
 [5.65890318 2.78748107 2.01472932 1.59934829]
 [2.22452534 1.88735317 1.52328405 1.24284272]
 [1.81315597 1.47838912 1.2324538  1.06050072]]


#### Copyright 2018, 2019, Stephen F Elston. All rights reserved.